# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение, чтобы привлечь новых клиентов. В нём можно будет узнать рыночную стоимость своего автомобиля. 
Нужно построить модель, которая умеет её определять. В распоряжении данные о технических характеристиках, комплектации и ценах других автомобилей.

 ### Критерии, которые важны заказчику:
 
   * качество предсказания;
    
   * время обучения модели;
    
   * время предсказания модели.
    
    
 ### Инструкция по выполнению проекта
 
   * Попробовать более простые модели — иногда они работают лучше. Эти редкие случаи легко пропустить, если всегда применять только бустинг. 

   * Сравнить характеристики моделей: время обучения, время предсказания, точность результата.


 ### Основные шаги:
 
   * Путь к файлу: /datasets/autos.csv.
    
   1) Изучить данные. Заполнить пропущенные значения и обработать аномалии в столбцах. Если среди признаков имеются неинформативные, удалить их.
    
   2) Подготовить выборки для обучения моделей.
    
   3) Обучить разные модели, одна из которых — LightGBM, как минимум одна — не бустинг. Для каждой модели попробовать разные гиперпараметры.
   
   4) Проанализировать время обучения, время предсказания и качество моделей.
    
   5) Опираясь на критерии заказчика, выбрать лучшую модель, проверить её качество на тестовой выборке.
    
    
 ### Примечания:
 
   * Для оценки качества моделей применить метрику RMSE.
    
   * Значение метрики RMSE должно быть меньше 2500.
    
   * Самостоятельно освоить библиотеку LightGBM и её средствами построить модели градиентного бустинга.
    
   * Время выполнения ячейки кода Jupyter Notebook можно получить специальной командой.
    
   * Модель градиентного бустинга может долго обучаться, поэтому изменять у неё только два-три параметра.
   
   
# Описание данных
Данные находятся в файле /datasets/autos.csv. 

   ### Признаки
 
   * DateCrawled — дата скачивания анкеты из базы
   
   * VehicleType — тип автомобильного кузова
   
   * RegistrationYear — год регистрации автомобиля
   
   * Gearbox — тип коробки передач
   
   * Power — мощность (л. с.)
   
   * Model — модель автомобиля
   
   * Kilometer — пробег (км)
   
   * RegistrationMonth — месяц регистрации автомобиля
   
   * FuelType — тип топлива
   
   * Brand — марка автомобиля
   
   * Repaired — была машина в ремонте или нет
   
   * DateCreated — дата создания анкеты
   
   * NumberOfPictures — количество фотографий автомобиля
   
   * PostalCode — почтовый индекс владельца анкеты (пользователя)
   
   * LastSeen — дата последней активности пользователя
   
   
### Целевой признак
 
   * Price — цена (евро)


In [1]:
import pandas as pd
import numpy as np
import seaborn as sb

from time import time

import lightgbm as lgb

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt

import warnings

In [2]:
try: 
    data = pd.read_csv('/Users/Sergej/Desktop/Practicum/Untitled Folder 1/autos.csv')
    
except: 
    data = pd.read_csv('/datasets/autos.csv')

### 1) Изучить данные. Заполнить пропущенные значения и обработать аномалии в столбцах. Если среди признаков имеются неинформативные, удалить их.

In [3]:
data

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


In [4]:
data.corr()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
Price,1.000000,0.026916,0.158872,-0.333199,0.110581,NaN,0.076055
RegistrationYear,0.026916,1.000000,-0.000828,-0.053447,-0.011619,NaN,-0.003459
Power,0.158872,-0.000828,1.000000,0.024002,0.043380,NaN,0.021665
Kilometer,-0.333199,-0.053447,0.024002,1.000000,0.009571,NaN,-0.007698
RegistrationMonth,0.110581,-0.011619,0.043380,0.009571,1.000000,NaN,0.013995
NumberOfPictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PostalCode,0.076055,-0.003459,0.021665,-0.007698,0.013995,NaN,1.000000


In [5]:
def snake(columns): 
    new_columns = []
    for word in columns: 
        snake_word = ''
        for i in range(len(word)-1):
            if word[i].isupper():
                snake_word += word[i].lower() 
            else:
                snake_word += word[i]
                
            if word[i+1].isupper():
                snake_word += '_'
                
        new_columns.append(snake_word+word[len(word)-1])
        
    return new_columns

In [6]:
# Приведем столбцы в змеиному регистру
data.columns = snake(data.columns)

Следует удалить стобцы: 'date_crawled', 'date_created', 'postal_code', 'last_seen', 'number_of_pictures'. Эти признаки важны для анализа, но не влияют на цену автомобиля.

In [7]:
data.drop(['date_crawled', 'date_created', 'postal_code', 'last_seen', 'number_of_pictures'], axis=1, inplace=True)

In [8]:
data

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354364,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes
354365,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               354369 non-null  int64 
 1   vehicle_type        316879 non-null  object
 2   registration_year   354369 non-null  int64 
 3   gearbox             334536 non-null  object
 4   power               354369 non-null  int64 
 5   model               334664 non-null  object
 6   kilometer           354369 non-null  int64 
 7   registration_month  354369 non-null  int64 
 8   fuel_type           321474 non-null  object
 9   brand               354369 non-null  object
 10  repaired            283215 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


In [10]:
numerical = ['price', 'registration_year', 'power', 'kilometer', 'registration_month']
categorial = list(set(data.columns) - set(numerical))

Обзор количественных признаков

In [11]:
for item in numerical: 
    print('_'*50)
    print(item)
    print(data[item].describe())
    print('NaN:', data[item].isna().sum())
    print('_'*50)

__________________________________________________
price
count    354369.000000
mean       4416.656776
std        4514.158514
min           0.000000
25%        1050.000000
50%        2700.000000
75%        6400.000000
max       20000.000000
Name: price, dtype: float64
NaN: 0
__________________________________________________
__________________________________________________
registration_year
count    354369.000000
mean       2004.234448
std          90.227958
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64
NaN: 0
__________________________________________________
__________________________________________________
power
count    354369.000000
mean        110.094337
std         189.850405
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: power, dtype: float64
NaN: 0
______________________________________________

In [12]:
data = data.query('registration_year < 2023 & price > 200 & power < 500')

Обзор категориальных признаков

In [13]:
for item in categorial: 
    print('_'*50)
    print(item)
    print(data[item].describe())
    print('NaN:', data[item].isna().sum(), '|', round(data[item].isna().sum() / len(data) * 100, 2), '%')
    print('_'*50)

__________________________________________________
gearbox
count     319207
unique         2
top       manual
freq      254974
Name: gearbox, dtype: object
NaN: 15207 | 4.55 %
__________________________________________________
__________________________________________________
brand
count         334414
unique            40
top       volkswagen
freq           72547
Name: brand, dtype: object
NaN: 0 | 0.0 %
__________________________________________________
__________________________________________________
repaired
count     273009
unique         2
top           no
freq      241803
Name: repaired, dtype: object
NaN: 61405 | 18.36 %
__________________________________________________
__________________________________________________
fuel_type
count     307781
unique         7
top       petrol
freq      204949
Name: fuel_type, dtype: object
NaN: 26633 | 7.96 %
__________________________________________________
__________________________________________________
model
count     318125
uniq

Заменим пропуски в категориальных признаках модой

In [14]:
for item in categorial:
    top = data[item].copy().describe()['top']
    data[item].fillna(top, inplace=True);

/Users/Sergej/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [15]:
data

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired
0,480,sedan,1993,manual,0,golf,150000,0,petrol,volkswagen,no
1,18300,coupe,2011,manual,190,golf,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354363,1150,bus,2000,manual,0,zafira,150000,3,petrol,opel,no
354365,2200,sedan,2005,manual,0,golf,20000,1,petrol,sonstige_autos,no
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


Данные готовы для дальнейшей работы. Количество машин с мощностью "0" составляет почти 10% от общего числа авто. Возможно, машины продаются без двигателя, или это способ обозначения пропусков.

###  2) Подготовить выборки для обучения моделей.

In [16]:
RND = 123
level = 2500
target='price'

numerical = ['registration_year', 'power', 'kilometer', 'registration_month']
categorial = ['vehicle_type', 'brand', 'repaired', 'gearbox', 'fuel_type', 'model']

Разделение датафрейма на обучающую, валидационную и тестовую выборки

In [17]:
def split_df(data, train, valid, test):
    df1, df_test = train_test_split(data, train_size=train + valid, random_state=RND)
    train /= (train + valid)
    df_train, df_valid = train_test_split(df1, train_size=train, random_state=RND)
    
    return df_train.reset_index(drop=True), df_valid.reset_index(drop=True), df_test.reset_index(drop=True)

In [18]:
df_train, df_valid, df_test = split_df(data, 0.5, 0.25, 0.25)

In [19]:
df_train

,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired
0,6600,other,2005,manual,190,rx_reihe,80000,6,petrol,mazda,no
1,14500,coupe,2000,manual,170,3er,150000,7,petrol,bmw,no
2,18999,coupe,2011,manual,184,3er,125000,6,gasoline,bmw,no
3,300,coupe,1992,manual,140,3er,150000,3,petrol,bmw,no
4,555,sedan,1996,manual,122,c_klasse,125000,6,petrol,mercedes_benz,no
...,...,...,...,...,...,...,...,...,...,...,...
167201,3200,small,2003,auto,61,golf,125000,12,petrol,smart,no
167202,750,small,1996,manual,60,polo,150000,6,petrol,volkswagen,no
167203,13500,sedan,2006,manual,218,3er,100000,1,petrol,bmw,no
167204,4999,sedan,2003,manual,101,golf,150000,4,gasoline,volkswagen,no


### 3. Обучить разные модели, одна из которых — LightGBM, как минимум одна — не бустинг. Для каждой модели попробовать разные гиперпараметры.

In [20]:
X_train, y_train = df_train.drop(target, axis=1), df_train[target]
X_valid, y_valid = df_valid.drop(target, axis=1), df_valid[target]
X_test, y_test = df_test.drop(target, axis=1), df_test[target]

**OneHotEncoder**

In [21]:
def get_OHE_train(df, X_train, categorial): 
    ohe = OneHotEncoder(drop='first')
    ohe.fit(df[categorial])
    encode_data = pd.DataFrame(ohe.transform(X_train[categorial]).toarray())
    
    x1 = X_train.reset_index(drop=True).drop(categorial, axis=1).copy()
    x1 = x1.join(encode_data)
    return x1, ohe

def get_OHE_test(X_test, categorial, ohe): 
    encode_data = pd.DataFrame(ohe.transform(X_test[categorial]).toarray())
    
    x1 = X_test.reset_index(drop=True).drop(categorial, axis=1).copy()
    x1 = x1.join(encode_data)
    return x1

**OrdinalEncoder**

In [22]:
def get_ORD_train(X_train, categorial): 
    enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    data = X_train.copy()
    enc.fit(data[categorial])

    data[categorial] = enc.transform(data[categorial])
    data.reset_index(drop=True, inplace=True)
    
    return data, enc

def get_ORD_test(X_test, categorial, enc): 
    data = X_test.copy()

    data[categorial] = enc.transform(data[categorial])
    data.reset_index(drop=True, inplace=True)
    return data

**StandardScaler + Encoder**

**OHE**

In [23]:
X_train_OHE, OHE = get_OHE_train(data, X_train, categorial)
X_valid_OHE = get_OHE_test(X_valid, categorial, OHE)
X_test_OHE = get_OHE_test(X_test, categorial, OHE)

In [24]:
ohe_scaler = StandardScaler()
ohe_scaler.fit(X_train_OHE[numerical])
X_valid_OHE[numerical] = ohe_scaler.transform(X_valid_OHE[numerical])
X_test_OHE[numerical] = ohe_scaler.transform(X_test_OHE[numerical])
X_train_OHE[numerical] = ohe_scaler.transform(X_train_OHE[numerical])

**OrdinalEncoder**

In [25]:
X_train_ORD, ORD = get_ORD_train(X_train, categorial)
X_valid_ORD = get_ORD_test(X_valid, categorial, ORD)
X_test_ORD = get_ORD_test(X_test, categorial, ORD)

In [26]:
ord_scaler = StandardScaler()
ord_list = numerical + categorial
ord_scaler.fit(X_train_ORD[ord_list])

X_valid_ORD[ord_list] = ord_scaler.transform(X_valid_ORD[ord_list])
X_test_ORD[ord_list] = ord_scaler.transform(X_test_ORD[ord_list])
X_train_ORD[ord_list] = ord_scaler.transform(X_train_ORD[ord_list])

**3.1) LightGBM**

In [27]:
warnings.filterwarnings("ignore")

In [28]:
params = {}
params['num_leaves'] = range(10, 50, 4)
params['max_bin'] = range(100, 1000, 100)

In [29]:
best_params = {'num_leaves': 'auto', 
               'max_bin': 100, 
               'time_learning': 0,
               'time_predict': 0,
               'RMSE': 1e10}

Поиск оптимального параметра: `num_leaves`

In [30]:
for i in params['num_leaves']: 
    boost = lgb.LGBMRegressor(metric='mse', 
                              num_leaves=i,
                              num_iterations=200)
    learn_start = time()
    boost.fit(X_train_ORD, y_train)
    learn_finish = time()

    start_predict = time()
    y_pred = boost.predict(X_valid_ORD)
    finish_predict = time()
    
    time_learn = learn_finish - learn_start
    time_predict = finish_predict - start_predict
    
    rmse = MSE(y_valid, y_pred) ** 0.5
    if best_params['RMSE'] > rmse: 
        best_params['RMSE'] = rmse
        best_params['num_leaves'] = i
        best_params['time_learning'] = time_learn
        best_params['time_predict'] = time_predict

Поиск оптимального параметра `max_bin`

In [31]:
for i in params['max_bin']: 
    try:
        boost = lgb.LGBMRegressor(metric='mse', 
                                  max_bin=i,
                                  num_leaves=best_params['num_leaves'], 
                                  num_iterations=200)
        learn_start = time()
        boost.fit(X_train_ORD, y_train)
        learn_finish = time()
    except: 
        boost = lgb.LGBMRegressor(metric='mse', 
                                  max_bin=i,
                                  num_iterations=200)
        learn_start = time()
        boost.fit(X_train_ORD, y_train)
        learn_finish = time()
    start_predict = time()
    y_pred = boost.predict(X_valid_ORD)
    finish_predict = time()
    
    time_learn = learn_finish - learn_start
    time_predict = finish_predict - start_predict
    
    rmse = MSE(y_valid, y_pred) ** 0.5
    if best_params['RMSE'] > rmse: 
        best_params['RMSE'] = rmse
        best_params['max_bin'] = i
        best_params['time_learning'] = time_learn
        best_params['time_predict'] = time_predict

Поиск оптимального параметра `min_data_in_leaf`

In [32]:
num_it = 200
boost = lgb.LGBMRegressor(metric='mse', 
                          max_bin=best_params['max_bin'],
                          num_leaves=best_params['num_leaves'], 
                          num_iterations=num_it)

learn_start = time()
boost.fit(X_train_ORD, y_train)
learn_finish = time()

predict_start = time()
y_pred = boost.predict(X_valid_ORD)
predict_finish = time()

print('RMSE:', MSE(y_valid, y_pred) ** 0.5)
print('r2_score:', r2_score(y_valid, y_pred))
print('time_learn', learn_finish - learn_start)
print('time_predict', predict_finish - predict_start)

RMSE: 1664.3978120795227
r2_score: 0.8637912521269935
time_learn 0.7486889362335205
time_predict 0.22218799591064453


Модель LGBMRegressor показала достаточно низкий показатель RMSE. Также стоит отметить, что скорость обучения очень высокая, это дает явное преимущество перед моделью случайного леса.

**3.2) Обычная линейная регрессия**

In [33]:
model = LinearRegression()
start = time()
model.fit(X_train_OHE, y_train)
y_pred = model.predict(X_valid_OHE)
finish = time()
print('RMSE:', MSE(y_valid, y_pred) ** 0.5)
print('pred_learn_time (sec):',finish - start)

RMSE: 2859.495346156146
pred_learn_time (sec): 1.6508018970489502


Модель линейной регрессии не достигает нужного диапазона RMSE.

**3.3) Модель случайного леса**

In [34]:
forest_params={'max_depth':10,
               'max_features': 1.0,
               'min_samples_leaf':2,
               'min_samples_split':2,
               'RMSE':1e10}

выбор оптимального `min_samples_leaf`

In [35]:
for leaf in range(1, 10): 
    model = RandomForestRegressor(n_jobs=-1, 
                                  random_state=RND,
                                  n_estimators=4, 
                                  min_samples_leaf=leaf,
                                  min_samples_split=forest_params['min_samples_split'],
                                  max_depth=forest_params['max_depth'])
    model.fit(X_train_ORD, y_train)
    y_pred = model.predict(X_valid_ORD)
    rmse = MSE(y_valid, y_pred) ** 0.5
    if rmse < forest_params['RMSE']: 
        forest_params['RMSE'] = rmse
        forest_params['min_samples_leaf'] = leaf

выбор оптимального `min_samples_split`: 

In [36]:
for split in range(2, 10): 
    model = RandomForestRegressor(n_jobs=-1, 
                                  random_state=RND,
                                  n_estimators=4, 
                                  min_samples_leaf=forest_params['min_samples_leaf'],
                                  min_samples_split=split,
                                  max_depth=forest_params['max_depth'])
    model.fit(X_train_ORD, y_train)
    y_pred = model.predict(X_valid_ORD)
    rmse = MSE(y_valid, y_pred) ** 0.5
    if rmse < forest_params['RMSE']: 
        forest_params['RMSE'] = rmse
        forest_params['min_samples_split'] = split
        

выбор оптимального `max_features`

In [37]:
for alpha in list(np.linspace(0.1, 1.0, 10)): 
    model = RandomForestRegressor(n_jobs=-1, 
                                  random_state=RND,
                                  n_estimators=4, 
                                  min_samples_leaf=forest_params['min_samples_leaf'],
                                  min_samples_split=forest_params['min_samples_split'],
                                  max_features = alpha,
                                  max_depth=forest_params['max_depth'])
    model.fit(X_train_ORD, y_train)
    y_pred = model.predict(X_valid_ORD)
    rmse = MSE(y_valid, y_pred) ** 0.5
    if rmse < forest_params['RMSE']: 
        forest_params['RMSE'] = rmse
        forest_params['max_features'] = alpha
        

выбор оптимального `max_depth`

In [38]:
for depth in range(5, 16): 
    model = RandomForestRegressor(n_jobs=-1, 
                                  random_state=RND,
                                  n_estimators=4, 
                                  min_samples_leaf=forest_params['min_samples_leaf'],
                                  min_samples_split=forest_params['min_samples_split'],
                                  max_features = forest_params['max_features'],
                                  max_depth=depth)
    model.fit(X_train_ORD, y_train)
    y_pred = model.predict(X_valid_ORD)
    rmse = MSE(y_valid, y_pred) ** 0.5
    if rmse < forest_params['RMSE']: 
        forest_params['RMSE'] = rmse
        forest_params['max_depth'] = depth
        

проверка на большом количестве деревьев

In [39]:
model = RandomForestRegressor(n_jobs=-1, 
                              random_state=RND,
                              n_estimators=400, 
                              min_samples_leaf=forest_params['min_samples_leaf'],
                              min_samples_split=forest_params['min_samples_split'],
                              max_features = forest_params['max_features'],
                              max_depth=forest_params['max_depth'])
learn_start = time()
model.fit(X_train_ORD, y_train)
learn_finish = time()

predict_start = time()
y_pred = model.predict(X_valid_ORD)
predict_finish = time()


print('RMSE:', MSE(y_valid, y_pred) ** 0.5)
print('time_learn', learn_finish - learn_start)
print('time_predict', predict_finish - predict_start)

RMSE: 1687.3360779741317
time_learn 11.478461980819702
time_predict 0.634300947189331


Метрика RMSE для модели случайного леса показала результат, подходящий под условия, но она все равно меньше, чем у LGBM. К тому же, эта моджель обучалась намного дольше, чем LGBM.

### 4) Тестирование лучшей модели
Была выбрана модель градиентного бустинга как самая лучшая по метрике RMSE и времени обучения и предсказания. Увеличим количество итераций до 2000, это займет больше времени, но ошибка будет меньше.

In [40]:
learn_start = time()
boost.fit(X_train_ORD, y_train)
learn_finish = time()

predict_start = time()
y_pred = boost.predict(X_test_ORD)
predict_finish = time()

print('RMSE:', MSE(y_test, y_pred) ** 0.5)
print('r2_score:', r2_score(y_test, y_pred))
print('time_learn', learn_finish - learn_start)
print('time_predict', predict_finish - predict_start)

RMSE: 1684.8599782898207
r2_score: 0.859484506068176
time_learn 0.7462871074676514
time_predict 0.21583795547485352


In [41]:
y_constant = np.array([y_train.mean()] * len(y_test))
print('RMSE_const:', MSE(y_test, y_constant) ** 0.5)

RMSE_const: 4494.739148248321


# Выводы

В ходе проекта был обработаy датасет с информацией об автомобилях. Главной целью проекта было построить модель, которая будет предсказывать цену автомобиля изходя из его параметров.

Сравнивались три модели регрессии: обычная линейная регрессия, случайный лес, модель градиентного бустинга. 

   * С валидационной выборкой справились модели LGBM и случайного леса. Метрики RMSE оказались довольно близкими, однако модель LGBM обучалась и предсказывала сильно быстрее: 
        * Случайный лес: 
         * RMSE: 1687.92
         * time_learn 11.7 sec
         * time_predict 0.65 sec
         
        * LGBM: 
         * RMSE: 1664.27
         * time_learn 0.7 sec 
         * time_predict 0.2 sec
         
     Поэтому для итогового тестирования использовалась модель LGBM с параметрами:
     * 'num_leaves': 46,
     * 'max_bin': 400,
     * num_it = 2000
     
   * На тестовой выборке модель показала такие результаты:
       * LGBM_model: 
        * RMSE: 1685
        * r2_score: 0.86 
        * time_learn 0.73 sec
        * time_predict 0.2 sec 
        
       * const_model (mean): 
        * RMSE_const: 4494.75
        
Модель LGBM обучается и предсказывает быстро и предсказания удовлетворяют необходимой величине RMSE < 2500
         
         